**This notebook contains a function that fetches a SMILE string starting from a KEGG compound page. It also contains the associated unittests.**

This notebook contains the basis of a function(s) to take a PubChem ID number and fetch the associated SMILES string from PubChem.

It also contains code pieces to pull an SID from a KEGG webpage.

In [4]:
import pubchempy as pc
import requests
import re

from bs4 import BeautifulSoup


There are multiple identifier types for each chemical in PubChem. The two we are interacting with here are **SID** (substance ID) and **CID** (chemical ID). CID can be used to acces SMILES directly with PubChemPy. **KEGG does not have CID**, only SID. SID can be turned into CID from which SMILES can be found. 

### Get SMILES from CID and SID

In [20]:
# get SMILES directly from CID
for compound in pc.get_compounds('243'):
    print(compound.isomeric_smiles)

C1=CC=C(C=C1)C(=O)O


In [8]:
# get SMILES from SID through mapping to CID
substance = pc.Substance.from_sid('3480')
cid = substance.standardized_cid
compound = pc.get_compounds(cid)[0]
print(compound.isomeric_smiles)

C1=CC=C(C=C1)C(=O)O


In [ ]:
# for compound in pc.get_compounds('glucose', 'name'):
#    print(compound.cid, compound.isomeric_smiles, compound.smiles)

In [69]:
def user_input_to_smiles(input_cid):
    """Takes a PubChem CID input and outputs the associated SMILES."""
    for compound in pc.get_compounds(input_cid):
        print(compound.isomeric_smiles)

In [72]:
user_input_to_smiles(243)

C1=CC=C(C=C1)C(=O)O


In [ ]:
def test_user_input_to_smiles():
    # check that the input is an integer
    # check that the output is a string
    

---
### Get SID from KEGG url 

This currently works from the compound page. I have not seen if it can be pulled from the reaction page.


In [7]:
# url of the desired KEGG compound page
url = 'https://www.genome.jp/dbget-bin/www_bget?cpd:C00180'
# access the url
response = requests.get(url)

# turn the webpage into html
soup = BeautifulSoup(response.content, 'html.parser')

# find the link that contains 'pubchem'
sid = soup.find('a', href=re.compile('https://pubchem\.ncbi'))

# print the string that is displayed as the link
# (this is the SID, which works with pubchempy to get the SMILES)
print(sid.string)

3480


In [60]:
#%%writefile kegg_utils.py

import pubchempy as pc
import re
import requests

from bs4 import BeautifulSoup


def kegg_to_sid(url):
    # access the url
    response = requests.get(url)

    # turn the webpage into html
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the link that contains 'pubchem'
    sid = soup.find('a', href=re.compile(r'https://pubchem\.ncbi'))

    sid_string = sid.string

    return sid_string


def kegg_to_smiles(url):
    """Uses the KEGG compound page url to find the compound's PubChem SID, then to find the SMILES for that compound using the SID."""

    # access the url
    response = requests.get(url)

    # turn the webpage into html
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the link that contains 'pubchem'
    sid = soup.find('a', href=re.compile(r'https://pubchem\.ncbi'))

    substance = pc.Substance.from_sid(sid.string)
    cid = substance.standardized_cid
    compound = pc.get_compounds(cid)[0]

    print(compound.isomeric_smiles)

Overwriting kegg_utils.py


In [62]:
#%%writefile test_kegg_utils.py
import unittest

import kegg_utils


def test_kegg_to_sid():
    url_list = [
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00180',
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00587',
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00002']
    for url in url_list:
        sid_str = kegg_utils.kegg_to_sid(url)
        sid_str.isdigit(), 'SID contains characters other than numbers'
    return


def test_kegg_to_smiles():

    url = 'https://www.genome.jp/dbget-bin/www_bget?cpd:C00587'
    smiles = kegg_utils.kegg_to_sid(url)

    assert len(smiles) >= 1, 'SMILES string is very short. Check SMILES.'
    isinstance(smiles, str), 'SMILES not returned as string.'
    
    return

Overwriting test_kegg_utils.py
